In [5]:
from emc.workflows.correct import init_emc_wf
from pathlib import Path
import nipype.pipeline.engine as pe
from nipype.interfaces import utility as niu
from dipy.data import get_fnames
from dmriprep.workflows.dwi.util import init_dwi_reference_wf
from dmriprep.interfaces.vectors import CheckGradientTable

#fdata, fbvals, fbvecs = get_fnames("sherbrooke_3shell")
#fdata, fbvals, fbvecs = get_fnames("stanford_hardi")
fdata = '/home/dpys/Data/ds000206-download/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.nii.gz'
fbvals = '/home/dpys/Data/ds000206-download/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.bval'
fbvecs = '/home/dpys/Data/ds000206-download/sub-THP0001/ses-THP0001CCF1/dwi/sub-THP0001_ses-THP0001CCF1_acq-GD31_run-01_dwi.bvec'


ImportError: cannot import name 'ValidateImage' from 'niworkflows.interfaces.images' (/usr/local/anaconda3/lib/python3.8/site-packages/niworkflows-1.4.0rc5-py3.8.egg/niworkflows/interfaces/images.py)

In [ ]:
# Create a workflow that can be added as a node to
# `~dmriprep.workflows.dwi.base.init_dwi_preproc_wf` and (for now) envelops
# `~dmriprep.workflows.dwi.util.init_dwi_reference_wf`
def build_emc_workflow(fdata, fbvals, fbvecs, work_dir='/tmp'):

    wf_emc = pe.Workflow(name='emc_correction')

    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_files', 'in_bval',
                                                      'in_bvec']),
        name='inputnode')

    inputnode.inputs.dwi_file = fdata
    inputnode.inputs.in_bval = fbvals
    inputnode.inputs.in_bvec = fbvecs

    emc_wf_node = init_emc_wf('emc_wf')

    wf_emc.connect([
        (inputnode, emc_wf_node, [('dwi_file', 'meta_inputnode.dwi_file'),
                                  ('in_bval', 'meta_inputnode.in_bval'),
                                  ('in_bvec', 'meta_inputnode.in_bvec')])
    ])

    wf_emc.base_dir = work_dir

    cfg = dict(execution={'stop_on_first_crash': False,
                          'parameterize_dirs': True, 'crashfile_format': 'txt',
                          'remove_unnecessary_outputs': False,
                          'poll_sleep_duration': 0, 'plugin': 'MultiProc'})
    for key in cfg.keys():
        for setting, value in cfg[key].items():
            wf_emc.config[key][setting] = value

    return wf_emc

In [2]:
wf = build_emc_workflow(fdata, fbvals, fbvecs)
res = wf.run(plugin='MultiProc')

NameError: name 'build_emc_workflow' is not defined

In [ ]:
list(res.nodes())

[emc_correction.emc_wf.emc_vectors_node,
 emc_correction.emc_wf.extract_b0_node,
 emc_correction.emc_wf.split_b0s_node,
 emc_correction.emc_wf.split_dwis_node,
 emc_correction.emc_wf.match_transforms_node,
 emc_correction.emc_wf.b0_based_vector_transforms,
 emc_correction.emc_wf.b0_based_image_transforms,
 emc_correction.emc_wf.merge_b0s_node,
 emc_correction.emc_wf.b0_median,
 emc_correction.emc_wf.prune_b0s_from_dwis_node,
 emc_correction.emc_wf.uncorrect_model_images,
 emc_correction.emc_wf.merge_EMC_corrected_dwis_node,
 emc_correction.emc_wf.dwi_reference_wf.validate,
 emc_correction.emc_wf.dwi_reference_wf.extract_b0,
 emc_correction.emc_wf.dwi_reference_wf.reg_b0,
 emc_correction.emc_wf.dwi_reference_wf.pre_mask,
 emc_correction.emc_wf.dwi_reference_wf.rescale_b0,
 emc_correction.emc_wf.dwi_reference_wf.enhance_and_skullstrip_dwi_wf.n4_correct,
 emc_correction.emc_wf.dwi_reference_wf.enhance_and_skullstrip_dwi_wf.fixhdr_unifize,
 emc_correction.emc_wf.dwi_reference_wf.enhance_an